# Training Model

In [1]:
import tensorflow as tf
gpus = tf.config.experimental.list_physical_devices('GPU')
if gpus:
  try:
    for gpu in gpus:
      tf.config.experimental.set_memory_growth(gpu, True)
  except RuntimeError as e:
    print(e)

In [2]:
import pandas as pd
import numpy as np
import random 

### Importing Data

In [3]:
#Load Data
Data = pd.read_csv("data.csv")

In [5]:
# Convert to DataFrame
df = pd.DataFrame(Data)

In [6]:
# Shuffle Rows
df = df.sample(frac = 1) 

In [8]:
# Convert to np array
dataset = np.array(df)

### Splitting in data and labels

In [9]:
labels = dataset[:, 0] # Labels
data = dataset[:, 1:2501] # Pixel values

### Splitting in Train/Test

In [10]:
#Data Shape
data.shape

(13199, 2500)

In [11]:
#Train
data_train = data[0:10000, :]
data_train = data_train / 255.
#Test
data_test = data[10000:, :]
data_test = data_test / 255.

In [12]:
print(data_train.shape)
print(data_test.shape)

(10000, 2500)
(3199, 2500)


In [13]:
#Labels Shape
labels.shape

(13199,)

In [14]:
#Reshape
labels = labels.reshape(labels.shape[0], 1)
#Train
labels_train = labels[0:10000, :]
labels_train = labels_train.T
#Test
labels_test = labels[10000:, :]
labels_test = labels_test.T

In [15]:
print(labels_train)
print(labels_test)

[[1 5 7 ... 3 8 5]]
[[ 8  1  2 ...  5 10  8]]


### One-hot Encoding

In [16]:
from keras.utils import np_utils

In [17]:
y_train = np_utils.to_categorical(labels_train)
y_test  = np_utils.to_categorical(labels_test)

### Reshaping variables

In [18]:
train_y = y_train.reshape(y_train.shape[1], y_train.shape[2])
test_y = y_test.reshape(y_test.shape[1], y_test.shape[2])

In [19]:
train_x = data_train.reshape(data_train.shape[0], 50, 50, 1)
test_x = data_test.reshape(data_test.shape[0], 50, 50, 1)

In [20]:
train_x.shape

(10000, 50, 50, 1)

In [21]:
test_x.shape

(3199, 50, 50, 1)

### Model Building

In [22]:
from keras.models import Sequential
from keras.layers import Dense, Conv2D, Flatten, MaxPooling2D, Dropout
from keras.callbacks import ModelCheckpoint

In [23]:
# Number of classes
classes = 12

In [24]:
model = Sequential()
model.add(Conv2D(32, (5, 5), input_shape=(50, 50, 1), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2), strides=(2, 2), padding='same'))
model.add(Conv2D(64, (5, 5), activation='sigmoid'))
model.add(MaxPooling2D(pool_size=(5, 5), strides=(5, 5), padding='same'))
model.add(Flatten())
model.add(Dense(1024, activation='relu'))
model.add(Dropout(0.6))
model.add(Dense(classes, activation='softmax'))

### Compiling Model

In [25]:
# loss=categorical_crossentropy
# optimizer= adam

In [26]:
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [27]:
# Creating Checkpoint

In [28]:
filepath = "emojinator.h5"
checkpoint1 = ModelCheckpoint(filepath, monitor='val_acc', verbose=1, save_best_only=True, mode='max')
callbacks_list = [checkpoint1]

### Training Model

In [29]:
model.fit(train_x, train_y, validation_data=(test_x, test_y), epochs=25, batch_size=64, callbacks=callbacks_list)

Epoch 1/25
157/157 [==============================] - 8s 53ms/step - loss: 0.6481 - accuracy: 0.8059 - val_loss: 0.0116 - val_accuracy: 0.9975
Epoch 2/25
157/157 [==============================] - 9s 56ms/step - loss: 0.0180 - accuracy: 0.9957 - val_loss: 0.0016 - val_accuracy: 1.0000
Epoch 3/25
157/157 [==============================] - 8s 54ms/step - loss: 0.0043 - accuracy: 0.9993 - val_loss: 3.4788e-04 - val_accuracy: 1.0000
Epoch 4/25
157/157 [==============================] - 8s 49ms/step - loss: 0.0023 - accuracy: 0.9996 - val_loss: 0.0012 - val_accuracy: 1.0000
Epoch 5/25
157/157 [==============================] - 8s 51ms/step - loss: 0.0027 - accuracy: 0.9996 - val_loss: 7.9121e-05 - val_accuracy: 1.0000
Epoch 6/25
157/157 [==============================] - 8s 49ms/step - loss: 8.6288e-04 - accuracy: 1.0000 - val_loss: 4.9379e-05 - val_accuracy: 1.0000
Epoch 7/25
157/157 [==============================] - 8s 51ms/step - loss: 0.0011 - accuracy: 0.9998 - val_loss: 6.8104e-05 - 

### Evaluate model on test dataset

In [30]:
scores = model.evaluate(test_x, test_y, verbose=0)

In [31]:
print("Model Accuracy: %.2f%%" % (scores[1] * 100))

Model Accuracy: 100.00%


### Saving the model

In [32]:
model.save('emoji_detection.h5')